#タンパク質折りたたみ問題

量子アニーリングを用いた創薬関連のタンパク質折りたたみ問題がハーバード大学の先生によって2012年に発表されていました。そのタンパク質折りたたみ問題の論文を元に実際にwildqatで解いてみたいと思います。

##参考にする論文
natureに掲載されているこちらの論文をベースにします。

Finding low-energy conformations of lattice protein models by quantum annealing
Alejandro Perdomo-Ortiz, Neil Dickson, Marshall Drew-Brook, Geordie Rose & Alán Aspuru-Guzik
Scientific Reports volume 2, Article number: 571 (2012)
https://www.nature.com/articles/srep00571



##問題の概要とHPモデル、MJモデル

タンパク質を単純にモデル化をしてそれをイジングモデルモデルに落とし込むという試みです。

まずは、HPモデルというモデルを使用しています。

HPモデル
• アミノ酸をＨ（疎水性、非極性アミノ酸）とＰ(親水性、極性アミノ酸）のいずれかに分ける。
• Ｈは、水を嫌い、互いに引き付けあう

参考：HPモデルhttp://www.iba.t.u-tokyo.ac.jp/iba/AI/HP.pdf

Mijazawa-Jernigan (MJ) model
今回は単純化されたMJモデルを利用します。

##MJモデルのQUBOへの適用
用意された塩基列を特定の方向に回転させる操作をイジングモデルに対応させています。

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_5.png?raw=1">
引用：https://www.nature.com/articles/srep00571

今回使用する塩基列は、PSVKMAの配列で、
下記のように特定の塩基列が隣接すると安定状態になり、エネルギーがへります。このエネルギーの安定化を使っってコスト関数を最小化させることを狙います。

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_0.png?raw=1">

また、今回塩基列を全て一度に処理するのは難しいのでいくつかのパターンに分けます。

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_1.png?raw=1">

引用：https://www.nature.com/articles/srep00571

上記のようにすでにいくつかの折りたたまれたパターンから出発して安定状態を求めます。数が多くないので書き出すことができ、もっとも低いエネルギー安定状態は下記のようになります。それぞれのパターンに対して安定状態に到達できる形状が異なるようなので、どれか１つのschemeを取り上げて一番エネルギーの低い状態を立式から導き出したいと思います。

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_2.png?raw=1">

引用：https://www.nature.com/articles/srep00571

##コスト関数
今回のコスト関数は、

$$E_p = E_{onsite} + E_{px} + E_{ext}$$

となります。１項目はタンパク質の塩基列が重ならないという条件、２項目は塩基列同士の近接の相互作用のエネルギー関数、３項目は外部からの影響です。今回３項目は使用しないということなので、

$$E_p = E_{onsite} + E_{pw}$$

となります。

##モデル
今回、論文中にも触れられている実験３をやって見たいと思います。塩基列は、

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_3.png?raw=1">

この順番ですが、今回はある程度折りたたまれた状態で始まります。

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_4.png?raw=1">

この場合、PSKVまでは決まっていて、Mは下か左が決まっていますので、回転方向はPから順番に書いてみると、

$$010010q_10q_2q_3$$

となり、３量子ビットの式を解けばよいことになります。$01$は右、$00$は下、$10$は左、Mは下か左しかないので、$00$か$10$なので、$0$は決まっていて、残りの$q_1$から$q_3$をイジング問題で求めます。

コスト関数は、

$$E = -1-4q_3+9q_1q_3+9q_2q_3-16q_1q_2q_3$$

##３体問題の２体問題への分解
ここで、QUBOではそのままでは解けない３体問題がでてきます。この際には数学変換で２体問題へと分解をします。詳しくは他のチュートリアルを参照ください。

新しい量子ビット$q_4$を導入して、

$$q_4=q_1q_2$$

これにペナルティ項を導入するとコスト関数は、

$$E = -1-4q_3+9q_1q_3+9q_2q_3-16q_3q_4+\delta(3q_4+q_1q_2-2q_1q_4-2q_2q_4)$$

こちらをWildqatに入れて計算をして見ます。

##Wildqatへ実装
ここで上記のQUBOを実装します。
デルタの値を10として見てとくと、

In [1]:
!pip install blueqat

     |████████████████████████████████| 51kB 2.3MB/s 


In [2]:
import blueqat.opt as wq
a = wq.opt()
d = 10;
a.qubo = [[0,d,9,-2*d],[0,0,9,-2*d],[0,0,-4,-16],[0,0,0,3*d]]
a.sa()


[0, 0, 1, 0]

答えは$0010$となりました、QUBOで表現されたタンパク質の折りたたみは、

$0100100010$

という回転を表現することになり、下記のようなものがもっとも安定なものとなります。

<img src="https://github.com/Blueqat/Wildqat/blob/master/examples_ja/img/024_7.png?raw=1">